# Supervised model for Audit Scoring

In [13]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from dstools.mlutils.corp_tax_audit_unsupervised import add_abs_diffs
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
%store -r q_industries
%store -r q_audit_data_combined

In [3]:
q_industries.head()

,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster_ind,k-cluster_g,sumsum_gross_perc_glob,sumsum_gross_perc_ind,is_ccluster_ind,is_ccluster_g
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.500045,0.500000,0.545455,0.545455,0.0,1,0.085914,0.090909,1,1
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.500184,0.499954,0.545455,0.545455,0.0,1,0.586863,0.636364,1,1
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.500093,0.500047,0.545455,0.545455,0.0,1,0.467638,0.545455,1,1
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,0.499573,0.500190,0.545455,0.545455,1.0,2,0.241137,0.181818,0,0
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.804250,0.498629,0.545455,0.590909,0.0,1,0.023600,0.090909,1,1


In [4]:
q_audit_data_combined.columns

Index(['date', 'original_index', 'business_id', 'assessment_amount', 'naics_2',
       'entity_name', 'first_period', 'last_period', 'change', 'change+',
       'change-', 'nquarters', 'index', 'naics2', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind', 'k-cluster_ind',
       'k-cluster_g', 'sumsum_gross_perc_glob', 'sumsum_gross_perc_ind',
       'ischange+_outside_ccluster', 'ischange+OR-_outside_ccluster',
       'is_ccluster_ind', 'is_ccluster_g', 'ischange+_outside_ccluster_ind',
       'ischange+OR-_outside_ccluster_ind'],

# Naive Positive Change

In [5]:
q_audit_data_combined.shape

(6989, 46)

In [6]:
X = q_audit_data_combined[[ 'date','naics2','maxNumLoc', 
       'eff_tax%_perc_glob', 'eff_tax%_perc_ind', 
       'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind',
        'sumsum_gross_perc_glob', 'sumsum_gross_perc_ind'                 
                          ]].copy()

In [7]:
X.loc[:,'quarter'] = X['date'].dt.quarter

X.drop(columns=['date'], inplace=True)

In [8]:
# Constructing y
y = np.where(q_audit_data_combined['change+'], 1, np.where(q_audit_data_combined['change-'], 0, 0))
y

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

X_train_encoded = pd.get_dummies(X_train, columns=['naics2'])
X_test_encoded = pd.get_dummies(X_test, columns=['naics2'])
X_test_encoded

,maxNumLoc,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,sumsum_gross_perc_glob,...,quarter,naics2_23,naics2_33,naics2_42,naics2_45,naics2_53,naics2_54,naics2_56,naics2_61,naics2_81
6714,1.0,0.407067,0.322512,0.416922,0.432957,0.344873,0.496885,0.330275,0.503176,0.708129,...,3,0,0,0,0,0,0,0,0,1
3250,1.0,0.059233,0.086198,0.939711,0.914828,0.074543,0.948685,0.093381,0.915854,0.227114,...,3,1,0,0,0,0,0,0,0,0
7012,1.0,0.012255,0.035983,0.988460,0.965690,0.040543,0.959967,0.043515,0.943096,0.873366,...,3,0,0,1,0,0,0,0,0,0
11064,2.0,0.323429,0.572488,0.412356,0.325371,0.537088,0.507749,0.518946,0.528007,0.991022,...,2,0,0,1,0,0,0,0,0,0
3280,1.0,0.283547,0.440470,0.418526,0.421117,0.465437,0.504066,0.428711,0.503920,0.887137,...,3,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,1.0,0.006845,0.004651,0.995758,0.999003,0.106045,0.932228,0.069767,0.973422,0.942350,...,1,0,0,0,0,0,1,0,0,0
7501,1.0,0.005130,0.002725,0.994763,0.997275,0.836255,0.093309,0.786104,0.167575,0.945169,...,2,0,0,0,1,0,0,0,0,0
4699,1.0,0.029519,0.089141,0.970974,0.910859,0.048771,0.983710,0.042139,0.977310,0.855958,...,2,0,0,1,0,0,0,0,0,0
8055,1.0,0.032680,0.042511,0.966197,0.958972,0.097733,0.924837,0.104301,0.901137,0.822815,...,3,1,0,0,0,0,0,0,0,0


In [14]:
# Creating an instance of the LogisticRegression model
xgb_naive = xgb.XGBClassifier()

# Fitting the model on the training data
xgb_naive.fit(X_train_encoded, y_train)

# Predicting on the test data
y_pred = xgb_naive.predict(X_test_encoded)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7718168812589413


In [11]:
# Generate the classification report
report = classification_report(y_test, y_pred)

# Display the classification report
print("XGBoost - Classification Report")
print("Naive Model - Positive Change")
print(report)

XGBoost - Classification Report
Naive Model - Positive Change
              precision    recall  f1-score   support

           0       0.79      0.79      0.79       770
           1       0.75      0.75      0.75       628

    accuracy                           0.77      1398
   macro avg       0.77      0.77      0.77      1398
weighted avg       0.77      0.77      0.77      1398



# Feature Evaluation - Positive Change Naive

In [12]:
# Analysis of importance of the difference variables
# get importance
models = [xgb_naive]
model_names = ['xgb_naive']

for model, name in zip(models, model_names):
    print(name)
    model.fit(X_train_encoded, y_train)
    
    # Changing threshold to improve precision
    # Predict probabilities for the test data
    y_proba = model.predict_proba(X_test_encoded)

    # Adjust the decision threshold
    threshold = 0.50 # Set a higher threshold to increase precision
    y_pred = (y_proba[:, 1] >= threshold).astype(int)
    
    y_pred = model.predict(X_test_encoded)
    importance_coeficients = model.feature_importances_
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    feature_importance_df = pd.Series(
                                     importance_coeficients, 
                                     index=X_test_encoded.columns)

    # Output of all levels
    print("\n Model:", '\033[1m' + model.__class__.__name__ + '\033[0m')
    for i,v in enumerate(importance_coeficients):
        if v !=0:
            print(f"Feature: {i}, {X_test_encoded.columns[i]}, Score: {v:.2f}" )


    # Display of only features that impacted the model
    n_important_features = feature_importance_df.loc[feature_importance_df>0].shape[0]
    display(feature_importance_df.loc[feature_importance_df>0].hvplot(
            kind='barh', 
            height=500,
            width=800,
            title= f"{n_important_features} Features relevant for {name}. Precision: {precision:.2f}, Accuracy: {accuracy:.2f}"))
    
importance_df = pd.concat(
    [pd.Series(model.feature_importances_, index=X_test_encoded.columns, name=name) for model, name in zip(models, model_names)],
    axis=1
)

# Graph with comparison
if len(models)>1:
    bars = importance_df.hvplot.barh(#y='index', x=importance_df.columns, 
                                     stacked=False, 
                                     height=1500, width=800,
                                     legend='top_right')



    display(bars)
if len(models)==1:
    display(importance_df.sort_values(by=model_names[0], ascending=False))
else: display(importance_df)

xgb_naive

 Model: XGBClassifier
Feature: 0, maxNumLoc, Score: 0.08
Feature: 1, eff_tax%_perc_glob, Score: 0.03
Feature: 2, eff_tax%_perc_ind, Score: 0.03
Feature: 3, deduc2income_perc_glob, Score: 0.04
Feature: 4, deduc2income_perc_ind, Score: 0.03
Feature: 5, 4D_eff_tax%_perc_glob, Score: 0.02
Feature: 6, 4D_deduc2income_perc_glob, Score: 0.02
Feature: 7, 4D_eff_tax_perc_ind, Score: 0.02
Feature: 8, 4D_deduc2income_perc_ind, Score: 0.02
Feature: 9, sumsum_gross_perc_glob, Score: 0.03
Feature: 10, sumsum_gross_perc_ind, Score: 0.03
Feature: 11, quarter, Score: 0.01
Feature: 12, naics2_23, Score: 0.04
Feature: 13, naics2_33, Score: 0.07
Feature: 14, naics2_42, Score: 0.06
Feature: 15, naics2_45, Score: 0.04
Feature: 16, naics2_53, Score: 0.04
Feature: 17, naics2_54, Score: 0.16
Feature: 18, naics2_56, Score: 0.06
Feature: 19, naics2_61, Score: 0.11
Feature: 20, naics2_81, Score: 0.06


/opt/anaconda3/envs/core/lib/python3.9/site-packages/hvplot/converter.py:1224: FutureWarning: Having a non-string as a column name in a DataFrame is deprecated and will not be supported in Holoviews version 1.16.
  dataset = Dataset(data, self.indexes)


:Bars   [index]   (0)

,xgb_naive
naics2_54,0.164628
naics2_61,0.106399
maxNumLoc,0.075472
naics2_33,0.071802
naics2_81,0.061740
naics2_42,0.060317
naics2_56,0.057228
deduc2income_perc_glob,0.041078
naics2_53,0.040419
naics2_23,0.037873
